In [ ]:
import os
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
import shutil

# Setup download folder (ensure this points to your actual download folder)
download_folder = os.path.join(os.path.expanduser("~"), "Downloads")
destination_folder = r"C:\Users\I33238\Downloads\DN"  # Set to your desired destination folder

# Set up Chrome options for automatic download
chrome_options = Options()
chrome_options.add_experimental_option("prefs", {
    "download.default_directory": download_folder,  # Downloads will go here
    "download.prompt_for_download": False,  # Don't prompt for download
    "directory_upgrade": True
})

# Setup WebDriver (this automatically downloads the appropriate driver)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

# Read CSV with links (adjust path)
csv_file_path = r"C:\Users\I33238\Downloads\prefinal_download.csv" # Replace with your CSV file path
df = pd.read_csv(csv_file_path)

# Assuming the CSV has 'url' and 'name' columns (adjust if necessary)
for index, row in df.iterrows(): #.iloc[121:]
    files_before_download = os.listdir(download_folder)
    link = row['treeLossFiresAnnual']  # The URL column in your CSV
    #'naturalForest' 'treeCoverPlantations'

    if pd.isna(link) or link == "NA":
        continue
    else:
        new_file_name = ("China"+"_"+str(row["Regions"])+"_"+str(row["provinces"])+"_"+"TreeLossFiresAnnual"+"_"+"2001_2024")
        # The desired name for the downloaded file
        # Go to the URL and trigger the download
        driver.get(link)
        # Wait for the download to complete (give it a few seconds)
        time.sleep(5)  # You can adjust this time based on file size or complexity

        # Get the latest downloaded file by checking the folder before and after the download
        files_after_download = os.listdir(download_folder)
        downloaded_file = list(set(files_after_download) - set(files_before_download))

        if downloaded_file:
        # Most recent downloaded file (assumes it's the only file downloaded)
            downloaded_file = downloaded_file[0]
        
        # Wait until the file is fully downloaded
            while downloaded_file.endswith(".crdownload"):  # .crdownload is a temporary file extension
                time.sleep(1)
                downloaded_file = [f for f in os.listdir(download_folder) if f.endswith('.csv')][0]

        # Create the new path to rename and move the file
            new_file_path = os.path.join(destination_folder, f"{new_file_name}.csv")
        
        # Rename and move the file to the destination folder
            original_file_path = os.path.join(download_folder, downloaded_file)
            shutil.move(original_file_path, new_file_path)

            print(f"Downloaded and renamed file to: {new_file_path}")
        else:
            print(f"No file found after downloading from {link}")

    #break
# Close the browser
driver.quit()
